### **WINDOW FUNCTIONS**

**Definition:**  
A **window function** in SQL performs calculations across a set of table rows related to the current row. Unlike regular aggregate functions, window functions do not collapse the result set. They allow you to perform calculations across a subset of data (a "window") while retaining the individual row context. Window functions are particularly useful for tasks like running totals, ranking, and moving averages.

### **Key Concepts:**

1. **Window Function:** A function that computes an aggregate or calculation over a set of rows, but retains the individual row in the result.
2. **OVER() Clause:** The `OVER()` clause defines the "window" over which the window function operates. This clause can optionally include partitions (to divide the result set into groups) and an order (to define the sequence of rows within each partition).
3. **PARTITION BY:** Divides the result set into partitions to apply the window function to each partition separately.
4. **ORDER BY:** Defines the order in which the rows are processed within each partition.

---

### **Common Window Functions:**

1. **ROW_NUMBER()**  
   Assigns a unique sequential integer to rows within a partition of the result set.

2. **RANK()**  
   Assigns a rank to each row within a partition of the result set. If two rows have the same value, they receive the same rank, and the next row gets the rank after the tied ranks (with gaps).

3. **DENSE_RANK()**  
   Similar to `RANK()`, but without gaps. If two rows have the same rank, the next row gets the next rank without skipping.

4. **NTILE(n)**  
   Divides the result set into `n` buckets and assigns a number to each row indicating which bucket it belongs to.

5. **SUM(), AVG(), MIN(), MAX()**  
   Aggregate functions can be used as window functions to calculate running totals, averages, etc.

6. **LEAD() and LAG()**  
   Returns the value of a column from the next or previous row in the result set.

---

### **Syntax for Window Functions:**

```sql
SELECT column1, column2, 
       window_function() OVER (
           PARTITION BY column1
           ORDER BY column2
       ) AS window_result
FROM table_name;
```

- `window_function()`: The window function (e.g., `ROW_NUMBER()`, `RANK()`, `SUM()`).
- `PARTITION BY column1`: (Optional) Divides the result set into partitions to apply the window function to each partition.
- `ORDER BY column2`: (Optional) Specifies the order in which the window function is applied.
- `window_result`: Alias for the result of the window function.

---

### **Example using the Student Database:**

Let's assume we want to calculate the **rank** of students based on their grades and also calculate a **running total** of their credits earned from different courses.

#### 1. **ROW_NUMBER() Example:**

We will assign a unique sequential number to each student based on their grade in the `Grades` table, ordered by their grade in descending order.

```sql
SELECT s.StudentID, s.Name, g.Grade,
       ROW_NUMBER() OVER (ORDER BY g.Grade DESC) AS Rank
FROM Students s
JOIN Enrollments e ON s.StudentID = e.StudentID
JOIN Grades g ON e.EnrollmentID = g.EnrollmentID;
```

#### Explanation:
- `ROW_NUMBER() OVER (ORDER BY g.Grade DESC)`: Assigns a sequential number to each row, ordered by the `Grade` in descending order (highest grade gets rank 1).
- The result will show a unique rank for each student based on their grade.

#### Example Result:
| StudentID | Name    | Grade | Rank |
|-----------|---------|-------|------|
| 1         | Alice   | A     | 1    |
| 2         | Bob     | B     | 2    |
| 3         | Charlie | C     | 3    |

#### 2. **RANK() Example:**

We will use `RANK()` to assign a rank to students based on their grade. If two students have the same grade, they will get the same rank, and the next rank will skip a number.

```sql
SELECT s.StudentID, s.Name, g.Grade,
       RANK() OVER (ORDER BY g.Grade DESC) AS Rank
FROM Students s
JOIN Enrollments e ON s.StudentID = e.StudentID
JOIN Grades g ON e.EnrollmentID = g.EnrollmentID;
```

#### Example Result:
| StudentID | Name    | Grade | Rank |
|-----------|---------|-------|------|
| 1         | Alice   | A     | 1    |
| 2         | Bob     | B     | 2    |
| 3         | Charlie | B     | 2    |

Notice that Bob and Charlie have the same grade, so they share the same rank of 2.

#### 3. **SUM() as a Window Function Example (Running Total):**

Let's calculate the running total of credits for each student across the courses they have enrolled in.

```sql
SELECT s.StudentID, s.Name, c.CourseName, c.Credits,
       SUM(c.Credits) OVER (PARTITION BY s.StudentID ORDER BY e.EnrollmentDate) AS RunningTotal
FROM Students s
JOIN Enrollments e ON s.StudentID = e.StudentID
JOIN Courses c ON e.CourseID = c.CourseID;
```

#### Explanation:
- `SUM(c.Credits) OVER (PARTITION BY s.StudentID ORDER BY e.EnrollmentDate)`: Calculates the cumulative sum of credits for each student, ordered by the date of enrollment.

#### Example Result:
| StudentID | Name    | CourseName | Credits | RunningTotal |
|-----------|---------|------------|---------|--------------|
| 1         | Alice   | Math       | 3       | 3            |
| 1         | Alice   | Science    | 4       | 7            |
| 2         | Bob     | History    | 3       | 3            |
| 2         | Bob     | English    | 3       | 6            |

#### 4. **LEAD() and LAG() Example:**

- **LEAD()** allows you to access the next row’s value.
- **LAG()** allows you to access the previous row’s value.

Let's find the grade difference between each student and the next student.

```sql
SELECT s.StudentID, s.Name, g.Grade,
       LEAD(g.Grade) OVER (ORDER BY g.Grade DESC) AS NextGrade,
       LAG(g.Grade) OVER (ORDER BY g.Grade DESC) AS PreviousGrade
FROM Students s
JOIN Enrollments e ON s.StudentID = e.StudentID
JOIN Grades g ON e.EnrollmentID = g.EnrollmentID;
```

#### Example Result:
| StudentID | Name    | Grade | NextGrade | PreviousGrade |
|-----------|---------|-------|-----------|---------------|
| 1         | Alice   | A     | B         | NULL          |
| 2         | Bob     | B     | C         | A             |
| 3         | Charlie | C     | NULL      | B             |

Notice that the `LEAD()` function provides the next row’s value, and `LAG()` provides the previous row’s value.

---

### **Summary of Window Functions:**

- **ROW_NUMBER()**: Assigns a unique sequential integer to each row.
- **RANK()**: Assigns ranks with gaps in case of ties.
- **DENSE_RANK()**: Similar to `RANK()`, but no gaps in case of ties.
- **NTILE(n)**: Divides the result set into `n` equal parts.
- **LEAD() and LAG()**: Access values from the next or previous row.
- **Aggregate Functions (SUM(), AVG(), etc.)**: Used with the `OVER()` clause to calculate running totals, averages, etc., without collapsing the result set.

Let me know if you'd like further details or examples on any specific window function!